# EDA 2 - Time of Read Data

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm import tqdm

import sys
sys.path.append(os.path.abspath('..'))

from tools.data_reader import DataReader

In [ ]:
dr = DataReader()

read_train_params = {
    'path': '../data/train.csv',
    'train_path': '../data/train_images',
    'height': 256,
    'width': 1600, 
    'col': False,
    'sep':'[_,]'
}

gen = dr.read_train(**read_train_params)
count = dr.count('../data/train_images')

In [ ]:
for i in tqdm(gen):
    pass

In [ ]:
write_tfr_params = {
    'data_generator': gen,
    'count': count,
    'tfrpath': 'train',
    'haslabel': True,
    'shards': 1000,
    'compression': 'GZIP', 'c_level': None}

dr.write_tfr(**write_tfr_params)